In [1]:
import gymnasium as gym
from stable_baselines3 import SAC, PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

from stable_baselines3.common.evaluation import evaluate_policy

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward, contact_cost_weight, healthy_z_range, contact_force_range):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    contact_cost_weight = contact_cost_weight,
                    healthy_z_range=healthy_z_range,
                    contact_force_range=contact_force_range)
                   # render_mode='none')

In [3]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)           # Default circa 0.1; esploriamo da 0.05 a 0.2
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.5, 1.5)     # Default tipico è 1; esploriamo da 0.5 a 1.5
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.1, 1.0)               # Default tipico 0.5; esploriamo da 0.1 a 1.0
    healthy_reward = trial.suggest_float('healthy_reward', 0.5, 1.5)                   # Default tipico 1; esploriamo da 0.5 a 1.5
    
    # Parametri aggiuntivi per Ant-v5
    contact_cost_weight = trial.suggest_float('contact_cost_weight', 1e-4, 1e-3)  # Es. range intorno a 5e-4 come default
    healthy_z_lower = trial.suggest_float('healthy_z_lower', 0.1, 0.3)             # Per definire l'intervallo di altezze "sane"
    healthy_z_upper = trial.suggest_float('healthy_z_upper', 0.8, 1.2)
    contact_force_min = trial.suggest_float('contact_force_min', -1.0, -0.5)         # Modificabile se usi forze di contatto
    contact_force_max = trial.suggest_float('contact_force_max', 0.5, 1.0)

    # Crea l'ambiente passando tutti i parametri
    env = make_env(
        reset_noise_scale,
        forward_reward_weight,
        ctrl_cost_weight,
        healthy_reward,
        contact_cost_weight=contact_cost_weight,
        healthy_z_range=(healthy_z_lower, healthy_z_upper),
        contact_force_range=(contact_force_min, contact_force_max)
    )
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # ---------------------------
    # Iperparametri per il modello SAC
    # ---------------------------
    # Parametri di ottimizzazione
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    # Buffer di replay: tipicamente per SAC si usano grandi dimensioni
    #buffer_size = trial.suggest_int('buffer_size', 50000, 1000000, step=50000)
    batch_size = trial.suggest_categorical('batch_size', [512, 1024, 2048, 4096])  
    gamma = trial.suggest_float('gamma', 0.965, 0.98)
    # Tau per il soft update della target network (molto sensibile in SAC)
    tau = trial.suggest_float('tau', 0.005, 0.05)
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)
    # Frequenza e passi di aggiornamento: controllano quanti step di gradient descent effettuare
    train_freq = trial.suggest_int('train_freq', 1, 10)
    gradient_steps = trial.suggest_int('gradient_steps', 1, 10)

    # Iperparametro per penalizzare la varianza dei reward, utile per evitare soluzioni troppo instabili
    # variance_penalty_weight = trial.suggest_float('variance_penalty_weight', 0.0, 0.5)

    # Crea ed allena il modello SAC con i parametri ottimizzati
    model = SAC("MlpPolicy", env,
                learning_rate=learning_rate,
                buffer_size=5000,
                batch_size=batch_size,
                gamma=gamma,
                tau=tau,
                ent_coef=ent_coef,
                train_freq=train_freq,
                gradient_steps=gradient_steps,
                verbose=0)
    model.learn(total_timesteps=200000)

    # Valuta il modello su 200 episodi (200 è ottimale)
    episodes = 150
    # episode_rewards = []
    # for episode in range(episodes):
    #     obs = env.reset()
    #     done = False
    #     episode_reward = 0
    #     while not done:
    #         action, _states = model.predict(obs)
    #         obs, reward, done, info = env.step(action)
    #         episode_reward += reward
    #     episode_rewards.append(episode_reward)

    # # Calcola reward media e varianza
    # mean_reward = np.mean(episode_rewards)
    # reward_std = np.std(episode_rewards)

    # # Definisce l'obiettivo: massimizzare la reward media penalizzando la varianza
    # score = mean_reward - std_penalty_weight * reward_std

    # print(f'Mean is: {mean_reward}, Std is: {reward_std}\n')

    env.training = False # Setta l'environment in modalità di valutazione
    env.norm_reward = False # Disabilita la normalizzazione della reward. Questo è importante per valutare correttamente il modello.

    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=episodes)

    return mean_reward
# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-13 12:11:08,372] A new study created in memory with name: no-name-f2632875-2c2d-4ce7-8218-801302600245
/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_5446/364702298.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[W 2025-02-13 12:19:42,021] Trial 0 failed with parameters: {'reset_noise_scale': 0.13991900009415809, 'forward_reward_weight': 0.9979938379094674, 'ctrl_cost_weight': 0.30501112707553135, 'healthy_reward': 0.8485717124810314, 'contact_cost_weight': 0.0004872160283846294, 'healthy_z_lower': 0.10276407252107123, 'healthy_z_upper': 0.9445635622722539, 'contact_force_min': -0.9214128668240257, 'contact_force_max': 0.767765752974386, 'learning_rate': 2.0050640007024038e-05, 'batch_size': 512, 'gamma': 0.9695339231327027, 'tau'

KeyboardInterrupt: 